In [17]:
import numpy as np
import pandas as pd
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import importlib
import ast

import os
import sys

path_ = os.path.abspath(os.path.join('..', '00_utils'))
if path_ not in sys.path:
    sys.path.append(path_)
    
import analyse_results
reload_module = analyse_results.reload_module

Graphs = reload_module('analyse_results', 'Graphs')
Results = reload_module('analyse_results', 'Results')

In [18]:
fn = 'interim_results/pyomo_net_size.pkl'
with open(f'{fn}', 'rb') as file:
    pyomo_net_size = pickle.load(file)

pyomo_net_size

,mse_coll_ode,mse_coll_ode_test,times_elapsed,x_labels
1,"[0.028493110537301594, 0.030058402025387624, 0...","[0.19860489711734686, 0.4140464229789177, 0.00...","[5.208147048950195, 5.353545188903809, 3.27817...","[6, 16, 1]"
2,"[0.1498640320699178, 0.1575184510855925, 0.228...","[0.3017180111333611, 0.5394433823485928, 0.025...","[4.2694010734558105, 6.473681926727295, 5.9222...","[6, 32, 1]"
3,"[0.200513539398833, 0.22171638512854042, 0.038...","[0.3685086521401875, 3.059065916960904, 0.0315...","[17.096319913864136, 12.965358018875122, 14.81...","[6, 64, 1]"
0,"[0.35414941489441193, 0.05088750093378449, 0.6...","[0.1282596000672475, 2.8236418107373487, 0.058...","[23.12026596069336, 35.69369101524353, 53.4508...","[6, 128, 1]"


In [19]:
fn = 'interim_results/diffrax_net_size.pkl'
with open(f'{fn}', 'rb') as file:
    diffrax_net_size = pickle.load(file)

diffrax_net_size

,mse_diffrax,mse_diffrax_test,x_labels,times_elapsed
0,"[0.07620478275677495, 0.09278675140985426, 0.1...","[0.5376941220996303, 0.2476474088598866, 0.395...","[6, 16, 1]","[9.30220890045166, 14.22188401222229, 15.85521..."
1,"[0.014170084471637973, 0.026092865441633615, 0...","[0.26674506393814534, 0.10491367753006223, 0.2...","[6, 32, 1]","[11.137876033782959, 15.190810680389404, 19.08..."
2,"[0.007728837776768791, 0.014807100001473143, 0...","[0.2109382651025699, 0.0824444037401971, 0.271...","[6, 64, 1]","[13.671804904937744, 18.614086866378784, 23.57..."
3,"[0.004136865251746844, 0.01223400609695736, 0....","[0.11767859375523866, 0.06974568928288458, 0.0...","[6, 128, 1]","[21.359206914901733, 26.66003680229187, 34.584..."
4,"[0.2736071454018502, 0.13643812462371527, 0.23...","[0.9461452787804859, 0.3402833991882759, 0.536...","[6, 16, 16, 1]","[12.225907325744629, 17.01177215576172, 25.949..."
5,"[0.09768878334343657, 0.07237938262018219, 0.0...","[0.555175193894065, 0.23193189645114903, 0.123...","[6, 32, 32, 1]","[17.16407299041748, 24.410378217697144, 33.058..."


In [23]:
def filter_by_labels(source_df, reference_df, label_column='x_labels'):
    """
    Filters rows of source_df to only those where the label_column values are in reference_df.

    """
    if label_column not in source_df.columns or label_column not in reference_df.columns:
        raise ValueError(f"The specified label_column '{label_column}' must exist in both DataFrames.")

    # convert list to tuple for hashable type in the label_column
    source_df['label_copy'] = source_df[label_column].apply(tuple)
    reference_df['label_copy'] = reference_df[label_column].apply(tuple)

    # create a set of labels from reference_df for fast lookup
    labels_set = set(reference_df['label_copy'])

    # filter source_df where label_column values are in the set from reference_df
    filtered_df = source_df[source_df['label_copy'].isin(labels_set)]
    filtered_df.drop(columns=['label_copy'], inplace=True)

    return filtered_df

In [24]:
filter_by_labels(pyomo_net_size, diffrax_net_size, label_column='x_labels')

,mse_coll_ode,mse_coll_ode_test,times_elapsed,x_labels
1,"[0.028493110537301594, 0.030058402025387624, 0...","[0.19860489711734686, 0.4140464229789177, 0.00...","[5.208147048950195, 5.353545188903809, 3.27817...","([, 6, ,, , 1, 6, ,, , 1, ])"
2,"[0.1498640320699178, 0.1575184510855925, 0.228...","[0.3017180111333611, 0.5394433823485928, 0.025...","[4.2694010734558105, 6.473681926727295, 5.9222...","([, 6, ,, , 3, 2, ,, , 1, ])"
3,"[0.200513539398833, 0.22171638512854042, 0.038...","[0.3685086521401875, 3.059065916960904, 0.0315...","[17.096319913864136, 12.965358018875122, 14.81...","([, 6, ,, , 6, 4, ,, , 1, ])"
0,"[0.35414941489441193, 0.05088750093378449, 0.6...","[0.1282596000672475, 2.8236418107373487, 0.058...","[23.12026596069336, 35.69369101524353, 53.4508...","([, 6, ,, , 1, 2, 8, ,, , 1, ])"


In [20]:
diffrax_net_size[diffrax_net_size.x_labels in pyomo_net_size.x_labels]

TypeError: unhashable type: 'Series'

In [ ]:
diffrax_net_size_filterd = diffrax_net_size[diffrax_net_size.x_labels in pyomo_net_size.x_labels]

Graphs.plot_boxplots(pyomo_net_size.mse_diffrax, diffrax_net_size.mse_diffrax, pyomo_net_size.x_labels, 'Sequential methpd model (Diffrax) performance across Model Size Configurations', 'MSE')